In [1]:
# from TDD.TrDD.BDD import BDD, get_int_key, normalize, Find_Or_Add_Unique_table
# from functools import lru_cache,wraps, update_wrapper

# def hashed_args(func):
#     @wraps(func)
#     def wrapper(*args):
#         # print(*tuple(sorted(args)))
#         return func(*tuple(sorted(args, key= hash)))
    
#     update_wrapper(wrapper, func, ('cache_info', 'cache_clear','cache_parameters')) #繼承lru_cache的屬性
#     return wrapper
# @hashed_args #為了達成交換率，先排序輸入的兩個bdd
# @lru_cache(maxsize=None)
# def inner_mul(bdd1,bdd2):
#     """The contraction of two bdds, var_cont is in the form [[4,1],[3,2]]"""
#     global global_index_order 
#     '''original key'''
#     k1=bdd1.node.key
#     k2=bdd2.node.key
#     w1=bdd1.weight
#     w2=bdd2.weight
#     print('inner mul',k1, k2 ,w1 ,w2)

#     if k1==k2==-1:
#         weig=bdd1.weight * bdd2.weight
#         res=BDD(weig)        
#         if get_int_key(weig)==(0,0):
#             res.weight=0
#             return res
#         else:
#             res.weight=weig
#             return res        

#     if k1==-1:
#         if w1==0:
#             bdd=BDD(bdd1.node)
#             bdd.weight=0
#             return bdd

#         bdd=BDD(bdd2.node)
#         bdd.weight = w1 * w2
#         return bdd
            
#     if k2==-1:
#         if w2==0:
#             bdd=BDD(bdd2.node)
#             bdd.weight=0
#             return bdd        

#         bdd=BDD(bdd1.node)
#         bdd.weight=w1 * w2
#         return bdd
    
#     bdd1.weight=1
#     bdd2.weight=1
                
#     bdd=BDD(bdd2.node)
#     bdd.weight=0

#     if k1 == k2:
#         for succ1 in bdd1.node.successor:
#             for succ2 in bdd2.node.successor:
#                 temp_bdd1=BDD(succ1[2])
#                 temp_bdd1.weight=succ1[1]
#                 temp_bdd2=BDD(succ2[2])
#                 temp_bdd2.weight=succ2[1]            
#                 temp_res= mul(temp_bdd1,temp_bdd2)
#                 if not succ1[0]+succ2[0]==0:
#                     if succ1[0]+succ2[0]==2 and k1 % 2 == 0:
#                         temp_res1 = normalize( k1 + 1,[[2,-1,Find_Or_Add_Unique_table(-1)]])
#                         temp_res1 = mul(temp_res1,temp_res)
#                         temp_res = add(temp_res,temp_res1)
#                     else:
#                         temp_res=normalize(k1, [[succ1[0]+succ2[0],temp_res.weight,temp_res.node]])
#                 bdd=add(bdd,temp_res)
#     elif k1 <= k2:
#         the_successor=[]
#         for succ1 in bdd1.node.successor:
#             temp_bdd1=BDD(succ1[2])
#             temp_bdd1.weight=succ1[1]
#             temp_res=mul(temp_bdd1,bdd2)
#             the_successor.append([succ1[0],temp_res.weight,temp_res.node])
#             bdd=normalize(k1,the_successor)
#     else:
#         the_successor=[]
#         for succ2 in bdd2.node.successor:
#             temp_bdd2=BDD(succ2[2])
#             temp_bdd2.weight=succ2[1]
#             temp_res=mul(bdd1,temp_bdd2)
#             the_successor.append([succ2[0],temp_res.weight,temp_res.node])
#             bdd=normalize(k2,the_successor)        
            
#     bdd.weight=bdd.weight * w1 * w2
#     bdd1.weight=w1
#     bdd2.weight=w2
    
#     return bdd      


# def mul(bdd1, bdd2):
#     bdd=inner_mul(bdd1, bdd2)

#     return bdd

# def add(bdd1, bdd2):
#     bdd=BDD(1)
#     return bdd

In [2]:
import sympy as sp
from TDD.TrDD.BDD import Ini_BDD, get_bdd, mul3
s0, s1, s2, s3, s4 = sp.symbols('s0 s1 s2 s3 s4')
order=[]
for i in range(5):
    order.append('sin(s%i)'%i)
    order.append('cos(s%i)'%i)
Ini_BDD(order)
bdd1 = get_bdd(sp.sin(s0)*sp.sin(s1)*sp.sin(s2)*sp.sin(s3))
bdd2 = get_bdd(sp.sin(s1)*sp.sin(s2)*sp.sin(s3)*sp.sin(s4))
bdd3 = get_bdd(sp.sin(s2)*sp.sin(s4)*sp.sin(s0)*sp.sin(s1))

result=mul3(bdd1,bdd2)
print('1st time:',result,mul3.cache_info())
result=mul3(bdd2,bdd1)
print('2st time:',result,mul3.cache_info())
mul3.cache_clear()
print(' clear  :',mul3.cache_info())

BDD 1131  0 -1 (1+0j) 1
BDD 1131  0 0 (1+0j) (1+0j)
BDD 1131  -1 -1 (1+0j) (1+0j)
BDD 43 (1+0j)
BDD 1131  1 1 (-1+0j) (1+0j)
BDD 1131  0 1 (1+0j) (1+0j)
BDD 1131  1 -1 (1+0j) (1+0j)
BDD 1131  0 0 (1+0j) (1+0j)
BDD 1131  1 1 (1+0j) (1+0j)
1st time: 0j CacheInfo(hits=10, misses=8, maxsize=None, currsize=8)
2st time: 0j CacheInfo(hits=11, misses=8, maxsize=None, currsize=8)
 clear  : CacheInfo(hits=0, misses=0, maxsize=None, currsize=0)


In [3]:
bdd1.key_2_index

{}

1

In [5]:
s=sp.IndexedBase('s')

sym_order1=dict()
sym_order2=dict()
for i in range(4):
    if i <2 :
        sym_order2[s[i].name]=3-i
    else:
        sym_order1[s[i].name]=3-i


sym_order = dict()
sym_order.update(sym_order2)
sym_order.update(sym_order1)

def Reorderer(sym_order):
    def sort_item(item):
        return item[1]
    items=list(sym_order.items())
    items.sort(key=sort_item)
    return items

Reorderer(sym_order)


[('s[3]', 0), ('s[2]', 1), ('s[1]', 2), ('s[0]', 3)]

In [6]:
sym_order

{'s[0]': 3, 's[1]': 2, 's[2]': 1, 's[3]': 0}